In [ ]:
from pyspark import SparkContext, SparkConf
cf = SparkConf()
cf.set("spark.submit.deployMode","client")
sc = SparkContext.getOrCreate(cf)
from pyspark.sql import SparkSession
spark = SparkSession \
	    .builder \
	    .appName("TransitTrends- Taxi") \
	    .config("spark.some.config.option", "some-value") \
	    .getOrCreate()

In [2]:
from pyspark.sql.functions import lit

In [29]:
fhv_df = spark.read.parquet("/shared/yt/TAXI/FHV/fhv_tripdata_2022-12.parquet")
for i in range(1,13):
    for j in range(19,23):
        if (i != 12 and j != 22):
            if (i < 10):
                month = "0"+str(i)
            else:
                month = str(i)
            url = "/shared/yt/TAXI/FHV/fhv_tripdata_20"+str(j)+"-"+month+".parquet"
            temp = spark.read.parquet(url)
            fhv_df = fhv_df.union(temp)

In [33]:
fhv_df.printSchema()

root
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropOff_datetime: timestamp (nullable = true)
 |-- PUlocationID: double (nullable = true)
 |-- DOlocationID: double (nullable = true)
 |-- SR_Flag: double (nullable = true)
 |-- Affiliated_base_number: string (nullable = true)



In [30]:
fhv_df.count()

69748675

In [32]:
fhv_df.dropDuplicates().count()

69596534

In [34]:
fhv_df.createOrReplaceTempView("fhv")

In [38]:
spark.sql("SELECT * FROM fhv").show()

+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|dispatching_base_num|    pickup_datetime|   dropOff_datetime|PUlocationID|DOlocationID|SR_Flag|Affiliated_base_number|
+--------------------+-------------------+-------------------+------------+------------+-------+----------------------+
|              B00008|2022-12-01 00:30:00|2022-12-01 00:48:00|        null|        null|   null|                B00008|
|              B00009|2022-12-01 00:37:00|2022-12-01 00:53:00|        null|        null|   null|                B00009|
|              B00009|2022-12-01 00:30:00|2022-12-01 03:50:00|        null|        null|   null|                B00009|
|              B00037|2022-12-01 00:11:21|2022-12-01 00:48:01|        null|       100.0|   null|                B00037|
|              B00111|2022-12-01 00:46:35|2022-12-01 02:11:00|        null|        null|   null|                B03404|
|              B00112|2022-12-01 00:08:0

In [53]:
from pyspark.sql.functions import concat
import pyspark.sql.functions as F
fhv_df = fhv_df.withColumn("pickup_time",fhv_df["pickup_datetime"].cast("timestamp"))
fhv_df = fhv_df.withColumn("month", F.month(fhv_df["pickup_time"])).withColumn("year", F.year(fhv_df["pickup_time"]))

In [55]:
fhv_df.createOrReplaceTempView("fhv_time")

In [72]:
monthly_totals = spark.sql("SELECT month, year, count(month, year) AS Val FROM fhv_time GROUP BY month, year ORDER BY year DESC, month DESC")

In [78]:
monthly_totals.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43137)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:43137)

In [76]:
import pandas as pd
import matplotlib.pyplot as plt

In [77]:
monthly_totals['date'] = monthly_totals['month'].map(str)+ '-' +monthly_totals['year'].map(str)
monthly_totals['date'] = pd.to_datetime(monthly_totals['date'], format='%m-%Y').dt.strftime('%m-%Y')
fig, ax = plt.subplots()
plt.plot_date(df['date'], df['Val'])
plt.show()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:43137)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:43137)